In [1]:
path_xlsx_parte = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [2]:
import sys
sys.path.append('../_amigocloud')

In [3]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [5]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [7]:
df_lib = pd.DataFrame(data)

In [8]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,181,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
1,182,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
2,183,44,2024-07-11 11:53:35+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-09 18:29:35+00:00,2024-07-09 20:55:35+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 01,None
3,184,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
4,185,44,2024-07-11 11:53:35+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-09 18:29:35+00:00,2024-07-09 20:55:35+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 01,None


In [9]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[44, 45]

In [10]:
lista_id = list(set(df_lib['id']))
lista_id

[181, 182, 183, 184, 185]

In [11]:
nulos = df_lib.isnull().sum()
nulos

id                     0
idd                    0
fecha_registro         0
piloto_1               0
piloto_2               0
hora_inicio            0
hora_fin               0
labor                  0
temperatura            0
viento                 0
humedad                0
estado_de_operacion    0
dron                   0
obs                    5
dtype: int64

In [12]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [13]:
if null_check:
    df_lib = None
else:
    None

In [14]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,181,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
1,182,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
2,183,44,2024-07-11 11:53:35+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-09 18:29:35+00:00,2024-07-09 20:55:35+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 01,None
3,184,45,2024-07-11 11:55:01+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-07-09 17:55:01+00:00,2024-07-11 20:55:01+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 02,None
4,185,44,2024-07-11 11:53:35+00:00,EDUARDO REYES,EDUARDO REYES,2024-07-09 18:29:35+00:00,2024-07-09 20:55:35+00:00,LIBERACION,18,20,60,CONCLUIDO,MT 01,None


In [15]:
df_parte = pd.read_excel(path_xlsx_parte)

In [16]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OPERACIÓN DETENIDA POR,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD
0,NaN,2024-07-09 20:43:32,28.0,69,40149,PAZ REA JOSE ERNESTO,47,SANTA MARTHA,A7,10.696703,...,CONCLUIDO,MT 02,AURELIO GARCIA,AURELIO GARCIA,NaN,NaN,SIEMBRA 2023,1,26,35.0
1,NaN,NaT,NaN,62,5421,HUBBAUER HOYOS ENRIQUE,187,MATIAS--HABBAUER,L7.2,1.835936,...,NaN,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,2,26,NaN
2,NaN,NaT,NaN,62,5421,HUBBAUER HOYOS ENRIQUE,187,MATIAS--HABBAUER,L6,1.960231,...,NaN,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,3,26,NaN
3,NaN,NaT,NaN,62,5421,HUBBAUER HOYOS ENRIQUE,187,MATIAS--HABBAUER,L9.1,3.607907,...,NaN,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,4,26,NaN
4,NaN,NaT,NaN,50,13132,ZUMARAN MENACHO JORGE,2123,LAS CHACRAS PORTACHUELO,L1.6,0.386826,...,NaN,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,5,26,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,NaN,NaT,NaN,79,41594,AGROPECUARIA CAMPO DULCE S.R.L.,577,SAN ANTONIO--IAG,L1,48.404143,...,NaN,NaN,NaN,NaN,NaN,NaN,SOLICITUD,184,32,NaN
184,NaN,NaT,NaN,79,41594,AGROPECUARIA CAMPO DULCE S.R.L.,577,SAN ANTONIO--IAG,L5,12.004987,...,NaN,NaN,NaN,NaN,NaN,NaN,SOLICITUD,185,32,NaN
185,NaN,2024-07-10 22:20:55,28.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C8.5,0.503588,...,CONCLUIDO,MT 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,186,32,41.0
186,NaN,2024-07-10 22:25:38,28.0,124,1530,C.I.T.T.C.A,299,C.I.T.T.C.A,C17.1,1.584073,...,CONCLUIDO,MT 02,AURELIO GARCIA,AURELIO GARCIA,NaN,NaN,SOLICITUD,187,32,42.0


In [17]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [18]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_16128\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [19]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [20]:
lista_idd

[44, 45]

In [21]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 44
se proceso IDD: 45


In [22]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 181
se proceso ID: 182
se proceso ID: 183
se proceso ID: 184
se proceso ID: 185
